In [99]:
#spacy english en_core_web_trf

#Librerias a usar
import nltk
from bert_serving.client import BertClient
import spacy
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /home/alvaromontero/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [61]:
#Cargar libros
Book_list=["/mnt/d/Repositorios/Tecnologias_de_lenguaje_natural/P2/Practica3/divine_comedy.txt",'/mnt/d/Repositorios/Tecnologias_de_lenguaje_natural/P2/Practica3/Oddysey.txt'
           ,'/mnt/d/Repositorios/Tecnologias_de_lenguaje_natural/P2/Practica3/Illiad.txt','/mnt/d/Repositorios/Tecnologias_de_lenguaje_natural/P2/Practica3/Great_Gatsby.txt',
           '/mnt/d/Repositorios/Tecnologias_de_lenguaje_natural/P2/Practica3/Tom_Sawyer.txt','/mnt/d/Repositorios/Tecnologias_de_lenguaje_natural/P2/Practica3/Wuthering_Heights.txt']


Books_raw_file=[]
for i in range(len(Book_list)):
    with open(Book_list[i],'r',encoding='utf-8')as f:
        file=f.read()
        Books_raw_file.append(file)
        


PREPROCESAMIENTO


In [96]:
#Funciones

def eliminar_etiquetas(text):
    pattern = re.compile(r'<[^>]+>|\([^)]+\)|--[^\-]+--')
    doc = pattern.sub('', text)
    return doc

# Removemos apostrofes para quedarnos con palabras como cant couldnt o dont
def remover_apostrofes(text):
    pattern = r"\\?'"
    doc = re.sub(pattern, '', text)
    return doc
# Nos quedamos con solo alfabeto eliminando caracteres especiales    
def remover_especiales(text):
    pattern = r"[^a-z\s]"
    doc = re.sub(pattern, ' ', text)
    return doc
# Quitamos dobles espacios para tener texto espaciado solamente por un espacio por palabra
def dobles_espacios(text):
    pattern = r'\s+'
    doc = re.sub(pattern, ' ', text)
    return doc
def remove_stopwords(text):
    words = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in words if word not in stop_words]
    filtered_text = ' '.join(filtered_words)
    return filtered_text

In [107]:
#Vectorizacion tfidf
def vectorizar_tfidf(corpus):
    # TF-IDF
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(corpus)
    
    # palabras únicas
    feature_names_tfidf = vectorizer.get_feature_names_out()

    print("Tamaño de Palabras únicas:", len(feature_names_tfidf))
    return tfidf_matrix

In [72]:
#Quitar inicio de texto
deleted_lines_list=[134,336,1997,32,459,7]

for i,book in enumerate(Books_raw_file):
    lines = book.splitlines()  # Split the book text into lines
    modified_lines = lines[deleted_lines_list[i]:]  # Keep lines starting from the specified index
    modified_book = "\n".join(modified_lines)  # Join the remaining lines back into a string
    Books_raw_file[i]=modified_book


In [97]:
for i,book in enumerate(Books_raw_file):
    Books_raw_file[i]=book.lower()
    
print(Books_raw_file[0][:200])

canto i


in the midway of this our mortal life,
i found me in a gloomy wood, astray
gone from the path direct: and e’en to tell
it were no easy task, how savage wild
that forest, how robust and rough


Eliminacion de caracteres no importantes


In [105]:
from copy import deepcopy
books_clean_files=[]
books_clean_stops_file=[]
for i,book in enumerate(Books_raw_file):
    Text_aux=eliminar_etiquetas(book)
    Text_aux=remover_apostrofes(Text_aux)
    Text_aux=dobles_espacios(Text_aux)
    Text_aux=remover_especiales(Text_aux)
    
    books_clean_stops_file.append(deepcopy(Text_aux))#Por si el modelo no necesita quitar stopwords
    
    Text_aux=remove_stopwords(Text_aux)
    books_clean_files.append(Text_aux)#Texto sin stopwords
    print(Text_aux[:45]+'\n')

canto midway mortal life found gloomy wood as

book gods council minerva visit ithaca challe

iliad book argument contention achilles agame

younger vulnerable years father gave advice t

chapter tom answer tom answer gone boy wonder

chapter returned visit landlord solitary neig



Extracción de oraciones clave

1) TF-IDF

In [116]:
#TF-IDF NLTK
tfidf_matrix=vectorizar_tfidf(books_clean_files)


Tamaño de Palabras únicas: 24430


In [117]:
print(tfidf_matrix)

  (0, 16134)	0.000573076668833097
  (0, 9211)	0.0009659803341068473
  (0, 23317)	0.0007921176925078543
  (0, 16168)	0.0009659803341068473
  (0, 19689)	0.0009659803341068473
  (0, 6644)	0.0009659803341068473
  (0, 17032)	0.0013375203702983554
  (0, 3840)	0.0009659803341068473
  (0, 16946)	0.0007921176925078543
  (0, 5417)	0.0007921176925078543
  (0, 5226)	0.0006687601851491777
  (0, 1117)	0.0007921176925078543
  (0, 12486)	0.0006687601851491777
  (0, 118)	0.0006687601851491777
  (0, 22588)	0.0009659803341068473
  (0, 22674)	0.0009659803341068473
  (0, 11285)	0.0007921176925078543
  (0, 978)	0.0009659803341068473
  (0, 15212)	0.0009659803341068473
  (0, 21130)	0.0009659803341068473
  (0, 8448)	0.0006687601851491777
  (0, 22937)	0.0006687601851491777
  (0, 21508)	0.0007921176925078543
  (0, 22186)	0.0009659803341068473
  (0, 1576)	0.0007921176925078543
  :	:
  (5, 6001)	0.006587425108172797
  (5, 6891)	0.0193051135918628
  (5, 9504)	0.0480574104308074
  (5, 5997)	0.002370320210233074
  (5